<a href="https://colab.research.google.com/github/rosacarla/chatbot-baseado-em-regras-e-dados/blob/main/Agente_de_di%C3%A1logo_baseado_em_regras_e_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

># AGENTE DE DIÁLOGO HÍBRIDO BASEADO EM REGRAS E DADOS
>CURSO: Tecnólogo em Inteligência Artificial Aplicada  
>DISCIPLINA: Agentes Conversacionais  
>AUTORA: Carla Edila Silveira  
>OBJETIVO: construir um agente de diálogo que trará ocorrências sobre determinado tema  
> DATA: 05/09/2023
______________________________________________________________________

<body>
<center>
<img src="https://i.postimg.cc/0Q7ZcBm7/header.png" align="middle">
</center>
</body>

>## 1. Qual o contexto do projeto?
><p align="justify">Um agente de diálogo de <i>question answering</i> que, baseado em um corpus de texto sobre um assunto, traz informações mais relevantes de acordo com a consulta do usuário.</p>

>## 2. Quais ferramentas e técnicas adotar?
>*   **NLTK** - toolkit de PLN em Python
>*   **Expressões Regulares** - pacote de regex do Python
>*   **urllib e BeautifulSoup** - bibliotecas para obter dados de páginas HTML
>*   **scikit-learn** - pacote com funcionalidades de manipulação de dados e Machine Learning (serão utilizados TF-IDF e Similaridade de cosseno)



> ## 3. Construção do agente de diálogo
>A operação do agente será deste modo:
>1.   Recebe **entrada** do usuário
>2.   **Pré-processa** a entrada do usuário
>3.   Calcula a **similaridade** entre a entrada e as sentenças do corpus
>4.   Obtém a sentença **mais similar do corpus**
>5.   Mostra-a como **resposta** ao usuário  

>Antes destas etapas, será criado o corpus ao obter automaticamente dados da Wikipedia.


> ## 4. Importação de bibliotecas
> Importar pacote de expressões regulares do Python e acesso ao WordNet dado pelo NLTK.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('rslp')# Stemming em pt-br
from nltk.corpus import stopwords
nltk.download('stopwords')# Lista de stopwords
import numpy as np
import random
import string
import bs4 as bs
import urllib.request
import re
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


>## 5. Construção do corpus
><p align="justify">Será feito um <i>web-scraping</i> para obter os dados automaticamente da Wikipedia. Este processo deve ser executado só uma vez, e o arquivo salvo em forma de texto na máquina.</p>

In [2]:
# Busca pagina sobre o BRASIL. Para mudar o tema basta colocar o link para outra pagina.
# É possivel obter dados de páginas diferentes, basta definir uma lista de links e iterar sobre elas.
codigo_html = urllib.request.urlopen('https://pt.wikipedia.org/wiki/Brasil')
codigo_html = codigo_html.read()

# Processa codigo HTML lido
html_processado = bs.BeautifulSoup(codigo_html, 'lxml')

# Busca todos os paragrafos do texto
paragrafos = html_processado.find_all('p')

texto = ''

# Percorre paragrafos e concatena textos
for p in paragrafos:
  texto += p.text

# Normaliza texto para minusculas
texto = texto.lower()
texto[0:1000]

'coordenadas: 10° s 55° obrasil (localmente\xa0[bɾaˈziw][b]), oficialmente república federativa do brasil (?\xa0escutar),[7] é o maior país da américa do sul e da região da américa latina, sendo o quinto maior do mundo em área territorial (equivalente a 47,3% do território sul-americano), com 8\xa0510\xa0417,771 km²,[1][8][9] e o sétimo em população[10][11] (com 203 milhões de habitantes, em agosto de 2022).[12] é o único país na américa onde se fala majoritariamente a língua portuguesa e o maior país lusófono do planeta,[13] além de ser uma das nações mais multiculturais e etnicamente diversas, em decorrência da forte imigração oriunda de variados locais do mundo. sua atual constituição, promulgada em 1988, concebe o brasil como uma república federativa presidencialista,[7] formada pela união dos 26 estados, do distrito federal e dos 5\xa0570 municípios.[7][14][nota 1]\nbanhado pelo oceano atlântico, o brasil tem um litoral de 7\xa0491\xa0km[13] e faz fronteira com todos os outros paí

>## 6. Pré-processamento do corpus
> É necessário remover caracteres especiais do texto e dividí-lo em sentenças válidas.

In [3]:
texto = re.sub(r'\[[0-9]*\]', ' ', texto)
texto = re.sub(r'\s+', ' ', texto)
texto[0:1000]

'coordenadas: 10° s 55° obrasil (localmente [bɾaˈziw][b]), oficialmente república federativa do brasil (? escutar), é o maior país da américa do sul e da região da américa latina, sendo o quinto maior do mundo em área territorial (equivalente a 47,3% do território sul-americano), com 8 510 417,771 km², e o sétimo em população (com 203 milhões de habitantes, em agosto de 2022). é o único país na américa onde se fala majoritariamente a língua portuguesa e o maior país lusófono do planeta, além de ser uma das nações mais multiculturais e etnicamente diversas, em decorrência da forte imigração oriunda de variados locais do mundo. sua atual constituição, promulgada em 1988, concebe o brasil como uma república federativa presidencialista, formada pela união dos 26 estados, do distrito federal e dos 5 570 municípios. [nota 1] banhado pelo oceano atlântico, o brasil tem um litoral de 7 491 km e faz fronteira com todos os outros países sul-americanos, exceto chile e equador, sendo limitado a no

In [4]:
sentencas = nltk.sent_tokenize(texto, language='portuguese')
palavras = nltk.word_tokenize(texto, language='portuguese')
sentencas[10:15]

['o vínculo colonial foi rompido, de fato, quando em 1808 a capital do reino foi transferida de lisboa para a cidade do rio de janeiro, depois de tropas francesas comandadas por napoleão bonaparte invadirem o território português.',
 'em 1815, o brasil se torna parte de um reino unido com portugal.',
 'dom pedro i, o primeiro imperador, proclamou a independência política do país em 1822. inicialmente independente como um império, período no qual foi uma monarquia constitucional parlamentarista, o brasil tornou-se uma república em 1889, em razão de um golpe militar chefiado pelo marechal deodoro da fonseca (o primeiro presidente), embora uma legislatura bicameral, agora chamada de congresso nacional, já existisse desde a ratificação da primeira constituição, em 1824. desde o início do período republicano, a governança democrática foi interrompida por longos períodos de regimes autoritários, até um governo civil e eleito democraticamente assumir o poder em 1985, com o fim da ditadura mil

>## 7. Funções de pré-processamento de entrada do usuário
><p align="justify">Cria funções para pré-processar as entradas do usuário, com retirada de  pontuações e uso de Stemming nos textos, para que palavras similares sejam processadas igualmente pelo algoritmo (por ex., pedra e pedregulho teriam mesma forma léxica).</p>

In [5]:
# Define funcao que faz Stemming em todo texto
def stemming(tokens):
  stemmer = nltk.stem.RSLPStemmer()
  novotexto = []
  for token in tokens:
    novotexto.append(stemmer.stem(token.lower()))
  return novotexto

# Funcao que remove pontuacao
removePontuacao = dict((ord(punctuation), None) for punctuation in string.punctuation)

def preprocessa(documento):
  return stemming(nltk.word_tokenize(documento.lower().translate(removePontuacao), language='portuguese'))

In [6]:
# Conferir como fica um texto apos seu pré-processamento
preprocessa("Olá meu nome é Lucas, eu moro no Brasil, e você?")

['olá', 'meu', 'nom', 'é', 'luc', 'eu', 'mor', 'no', 'brasil', 'e', 'voc']

>## 8. Resposta a saudações
><p align="justify">Embora seja um sistema de diálogo baseado em tarefas, é provável que o usuário inicie conversas com saudações ao agente. Por isso, será desenvolvida uma função (regras) para tratar desta situação.</p>  
><p align="justify">Serão criadas algumas respostas possíveis, dentre as quais serão escolhidas algumas aleatoriamente, para evitar que o agente fique repetitivo.</p>

In [7]:
saudacoes_entrada = ("olá", "bom dia", "boa tarde", "boa noite", "oi", "como vai", "e aí", "tudo bem")
saudacoes_respostas = ["olá", "olá, espero que esteja tudo bem contigo", "oi", "Oie", "Seja bem-vindo, em que posso te ajudar?"]

def geradorsaudacoes(saudacao):
  for token in saudacao.split():
    if token.lower() in saudacoes_entrada:
      return random.choice(saudacoes_respostas)

In [12]:
# Ao executar este exemplo várias vezes serão vistas respostas diferenres
geradorsaudacoes('Olá')

'olá, espero que esteja tudo bem contigo'

>## 9. Resposta a consultas do usuário
><p align="justify">Função para tratar consultas do usuário, com a comparação da similaridade entre a entrada do usuário e as sentenças do corpus. Se houver, a sentença mais similar será mostrada como resposta.</p>
><p align="justify">Nesta função utilizam-se os algoritmos TD-IDF (<i>Term Frequency – Inverse Document Frequency</i>) e similaridade de cosseno.</p>

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
def geradorrespostas(entradausuario):
  resposta = ''
  sentencas.append(entradausuario)

  word_vectorizer = TfidfVectorizer(tokenizer=preprocessa, stop_words=stopwords.words('portuguese'))
  all_word_vectors = word_vectorizer.fit_transform(sentencas)
  similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
  similar_sentence_number = similar_vector_values.argsort()[0][-2]

  matched_vector = similar_vector_values.flatten()
  matched_vector.sort()
  vector_matched = matched_vector[-2]

  if vector_matched == 0:
    resposta = resposta + "Me desculpe, não entendi o que você pediu."
    return resposta
  else:
    resposta = resposta + sentencas[similar_sentence_number]
    return resposta

>## 10. Interação com o agente de diálogo
><p align="justify">Define um algoritmo que continue interagindo com o usuário até que ele decida finalizar.</p>  
><p align="justify">O resultado não é sempre o ideal, porém cobre muitas das possíveis perguntas. Se utilizássemos apenas regras de diálogo para responder perguntas sobre um assunto, precisaríamos de centenas de regras.</p>
><p align="justify"> Como as respostas são baseadas em dados, apenas uma regra que calcula similaridade com o corpus é suficiente. Faça perguntas como:</p>  

>*  *Qual o esporte mais popular no Brasil?*
>*  *Quais eventos esportivos o Brasil já organizou?*
>*  *Como é a cozinha brasileira?*
>*  *Onde são realizadas pesquisas tecnológicas no Brasil?*


In [ ]:
continue_dialogue = True
print("Olá, eu sou o Agente Tupiniquim. Me pergunte qualquer coisa sobre nosso país.")
while (continue_dialogue == True):
  # Obtem entrada do usuario
  human_text = input().lower()

  if human_text != 'tchau':
    if human_text == 'obrigado' or human_text == 'muito obrigado' or human_text == 'agradecido':
      continue_dialogue = False
      print("Agente Tupiniquim: Disponha")
    else:
      if geradorsaudacoes(human_text) != None:
        print("Agente Tupiniquim: " + geradorsaudacoes(human_text))
      else:
        print("Agente Tupiniquim: ", end="")
        print(geradorrespostas(human_text))
        sentencas.remove(human_text)
  else:
    continue_dialogue = False
    print("Agente Tupiniquim: Até a próxima.")

Olá, eu sou o Agente Tupiniquim. Me pergunte qualquer coisa sobre nosso país.
Qual é o esporte mais popular do Brasil?
Agente Tupiniquim: o futebol é o esporte mais popular no brasil.
Quais eventos esportivos o Brasil já organizou?
Agente Tupiniquim: no automobilismo, pilotos brasileiros ganharam o campeonato mundial de fórmula 1 oito vezes: emerson fittipaldi, em 1972 e 1974; nelson piquet, em 1981, 1983 e 1987; e ayrton senna, em 1988, 1990 e 1991. o circuito localizado em são paulo, autódromo josé carlos pace, organiza anualmente o grande prêmio do brasil de fórmula 1. o brasil já organizou eventos esportivos de grande escala: sediou as copas do mundo de 1950, na qual foi o vice-campeão, e 2014, quando ficou em quarto lugar.
Como é a cozinha brasileira?
Agente Tupiniquim: a cozinha brasileira varia muito de acordo com a região, refletindo a combinação de populações nativas e de imigrantes pelo país.
Onde são realizadas pesquisas tecnológicas no Brasil?
Agente Tupiniquim: a pesquisa 

>## 11. Como melhorar o projeto?
><p align="justify">Este agente de diálogo utiliza modelo baseado em regras, em que uma das regras usa corpus de dados para formular respostas. Desse modo, o modelo ficou mais flexível, sem necessidade de criar centenas/milhares de regras. Vejamos como melhorar o projeto:</p>  
><p align="justify">A) Além dos parágrafos (tag "p") da página da Wikipedia, podem ser usados dados dispostos na coluna direita, que trazem informações relevantes, como população, atual presidente etc., para montar sentenças.</p>  
><p align="justify">B) Melhorar o cálculo de similaridade com uso de um modelo de Word Embeddings, além do TF-IDF.</p>
><p align="justify">C) Obter dados sobre o Brasil a partir de diferentes fontes.</p>
><p align="justify">D) Criar classificador de contexto para o agente e, de modo dinâmico, buscar páginas da Wikipedia correspondentes à pergunta do usuário, para depois dar a resposta. Desse modo o agente não se limitaria a perguntas sobre o Brasil.</p>



>## 12. Referências e material complementar  
>* [Python for NLP: Creating a Rule-Based Chatbot](https://stackabuse.com/python-for-nlp-creating-a-rule-based-chatbot/)
>* [Building a Simple Chatbot from Scratch in Python (using NLTK)](https://morioh.com/p/6cc33336784c)
>* [Building a simple chatbot in python](https://medium.com/nxtplus/building-a-simple-chatbot-in-python-3963618c490a)
>* [Designing A ChatBot Using Python: A Modified Approach](https://towardsdatascience.com/designing-a-chatbot-using-python-a-modified-approach-96f09fd89c6d)
>* [Build Your First Python Chatbot Project](https://dzone.com/articles/python-chatbot-project-build-your-first-python-pro)
>* [Python Chatbot Project – Learn to build your first chatbot using NLTK & Keras](https://data-flair.training/blogs/python-chatbot-project/)
>* [Python Chat Bot Tutorial - Chatbot with Deep Learning (Part 1)](https://www.youtube.com/watch?v=wypVcNIH6D4)
>* [Intelligent AI Chatbot in Python](https://www.youtube.com/watch?v=1lwddP0KUEg)
>* [Coding a Jarvis AI Using Python 3 For Beginners](https://www.youtube.com/watch?v=NZMTWBpLUa4)  
>* Projeto elaborado a partir de Notebook criado por Prof. [Lucas Oliveira](http://lattes.cnpq.br/3611246009892500).